# Import

In [3]:
from run import *
import pandas as pd
from torch import nn

parser = get_parser()
argv = """
  --task_name classification \
  --data mimic \
  --result_path scratch \
  --use_gpu \
  --root_path ./dataset/mimic_iii/ \
  --data_path mimic_iii.pkl \
  --model DLinear \
  --flag train
""".split()
args = parser.parse_args(argv)
initial_setup(args)

In [4]:
if args.task_name == 'classification': Exp = Exp_Classification
else: Exp = Exp_Long_Term_Forecast
exp = Exp(args)  # set experiments
_, dataloader = exp._get_data(args.flag)

Use GPU: cuda:0
train 18390
test 2299
Experiments will be saved in scratch\mimic_iii_DLinear
train 18390


In [11]:
import pickle

filepath = os.path.join(args.root_path, args.data_path)
with open(filepath, 'rb') as input_file:
    data = pickle.load(input_file)

X = np.array([row[0] for row in data])
# batch x features x seq_len -> batch x seq_len x features
X = X.transpose((0, 2, 1))



In [13]:
Y = np.array([int(row[1]) for row in data])
labels = pd.Series(Y, dtype='category')
class_names = labels.cat.categories
labels_df = pd.DataFrame(labels.cat.codes,
                            dtype=np.int8)

In [5]:
batch_x, label, padding_mask = next(iter(dataloader))
batch_x = batch_x.float().to(exp.device)
padding_mask = padding_mask.float().to(exp.device)

In [ ]:
batch_x, batch_y, batch_x_mark, batch_y_mark = next(iter(dataloader))
batch_x = batch_x.float().to(exp.device)
batch_y = batch_y.float().to(exp.device)

batch_x_mark = batch_x_mark.float().to(exp.device)
batch_y_mark = batch_y_mark.float().to(exp.device)
# decoder input
dec_inp = torch.zeros_like(batch_y[:, -exp.args.pred_len:, :]).float()
dec_inp = torch.cat([batch_y[:, :exp.args.label_len, :], dec_inp], dim=1).float()

In [7]:
_ = exp.model.eval()

In [8]:
output = exp.model(batch_x, padding_mask, None, None)

# RNN

In [5]:
model = LSTM(args)
model = model.to(exp.device).float()

In [6]:
output = model(batch_x)
output.shape

torch.Size([32, 1])

# TCN

In [73]:
from torch.nn.utils import weight_norm


class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size
        
    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()
    
class Reshape(nn.Module):
    def __init__(self, *shape):
        super(Reshape, self).__init__() 
        self.shape = shape
    def forward(self, x):
        return x.reshape(x.shape[0], -1) if not self.shape else x.reshape(-1) if self.shape == (-1,) else x.reshape(x.shape[0], *self.shape)
    def __repr__(self): return f"{self.__class__.__name__}({', '.join(['bs'] + [str(s) for s in self.shape])})"
    
    
class GAP1d(nn.Module):
    "Global Adaptive Pooling + Flatten"
    def __init__(self, output_size=1):
        super(GAP1d, self).__init__() 
        self.gap = nn.AdaptiveAvgPool1d(output_size)
        self.flatten = Reshape()
    def forward(self, x):
        return self.flatten(self.gap(x))

class TemporalBlock(nn.Module):
    def __init__(self, ni, nf, ks, stride, dilation, padding, dropout=0.):
        super(TemporalBlock, self).__init__() 
        self.conv1 = weight_norm(nn.Conv1d(ni,nf,ks,stride=stride,padding=padding,dilation=dilation))
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)
        self.conv2 = weight_norm(nn.Conv1d(nf,nf,ks,stride=stride,padding=padding,dilation=dilation))
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)
        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1, 
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(ni,nf,1) if ni != nf else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None: self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)

def TemporalConvNet(c_in, layers, ks=2, dropout=0.):
    temp_layers = []
    for i in range(len(layers)):
        dilation_size = 2 ** i
        ni = c_in if i == 0 else layers[i-1]
        nf = layers[i]
        temp_layers += [TemporalBlock(ni, nf, ks, stride=1, dilation=dilation_size, padding=(ks-1) * dilation_size, dropout=dropout)]
    return nn.Sequential(*temp_layers)

class TCN(nn.Module):
    def __init__(self, configs, c_in, c_out, layers=8*[25], ks=7, conv_dropout=0., fc_dropout=0.):
        super(TCN, self).__init__()
        self.configs = configs 
        self.task_name = configs.task_name
        
        self.tcn = TemporalConvNet(c_in, layers, ks=ks, dropout=conv_dropout)
        self.dropout = nn.Dropout(fc_dropout) if fc_dropout else None
        self.gap = GAP1d()
        
        if self.task_name == 'classification':
            self.linear = nn.Linear(layers[-1], configs.num_class)
        else:
            self.linear = nn.Linear(layers[-1], configs.pred_len*c_out)
        
        self.init_weights()

    def init_weights(self):
        self.linear.weight.data.normal_(0, 0.01)

    def forward(self, x):
        # batch x seq_len x features -> batch x features x seq_len
        x = x.permute(0, 2, 1)
        # batch x hidden_size x seq_len
        x = self.tcn(x)
        
        # pools over the second dimension. 
        # [dim 0, (gap_output size x dim 2)]
        x = self.gap(x)

        if self.dropout is not None: x = self.dropout(x)
        x = self.linear(x)
        if self.task_name == 'classification':
            return x # [B, N]
        else:
            x = x.reshape((-1, self.configs.pred_len, self.configs.c_out))
            f_dim = -1 if self.configs.features == 'MS' else 0
            return x[:, :, f_dim:]
        return 

In [58]:
tcn = TemporalConvNet(c_in=7, layers=8*[25], ks=10, dropout=0).to(exp.device)

In [59]:
x = tcn(batch_x.permute(0, 2, 1))

In [69]:
gap = GAP1d(output_size=1).to(exp.device)
x_gap = gap(x.permute(0, 2, 1))

In [74]:
linear = nn.Linear(args.seq_len, args.c_out).to(exp.device)

In [75]:
x_linear = linear(x_gap)
x_linear.shape

torch.Size([32, 7])

In [ ]:
linear2 = nn.Linear(args.seq_len, args.pred_len*args.c_out).to(exp.device)

In [53]:
model = TCN(configs=args, c_in=7, c_out=1)
model = model.to(exp.device).float()

In [16]:
output = model(batch_x)
output.shape